In [1]:
import numpy as np
import pandas as pd
import uproot

### Srotolo i tree e faccio tre singoli file 

In [2]:
fname = 'analysis.root'
fileIn = fname
filesOut= []

sgn_treename = 'signal_bbA_MA300tree'
bkgttbar_treename = 'bkg_ttbar_nlotree'
bkgDY_treename = 'bkg_DY_nlo1tree'
hood = [sgn_treename, bkgttbar_treename, bkgDY_treename]

for tree in hood:
    print ("Reading trees {} in file {}".format(tree, fileIn))
    data = uproot.open(fileIn)[tree]
    names = data.keys()
    
    #data.arrays(names)
    out = pd.DataFrame.from_dict(data.arrays(names))
    outfilename = tree + '.csv'
    out.to_csv(outfilename)
    filesOut.append(outfilename)
    print ('created csv file' + outfilename)

Reading trees signal_bbA_MA300tree in file analysis.root
created csv filesignal_bbA_MA300tree.csv
Reading trees bkg_ttbar_nlotree in file analysis.root
created csv filebkg_ttbar_nlotree.csv
Reading trees bkg_DY_nlo1tree in file analysis.root
created csv filebkg_DY_nlo1tree.csv


#### Aggiungo la colonna con label

In [9]:
import csv

for file in filesOut:
    reader = csv.reader(open(file, 'r'))
    writer = csv.writer(open("__" + file, 'w'))

    headers = next(reader)
    headers.append("label")

    writer.writerow(headers)
    for row in reader:
        if file == "signal_bbA_MA300tree.csv":
            row.append("signal")
            writer.writerow(row)
        else:
            row.append("background")
            writer.writerow(row)

### Unisco i tre file

In [ ]:
import os
import glob


os.chdir('/home/william/Tesi/')

all_filenames = [file for file in glob.glob('__*')]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "analysis.csv", index=False, encoding='utf-8-sig')